In [ ]:
########################################
# 1. under/over/critical damping RK4   #
########################################




In [ ]:
########################################
# 2. resonance solution RK4            #
########################################

In [ ]:
########################################
# 3. pygame                            #
########################################

